In [5]:
import time
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

edge_driver_path = r"C:\Users\admin\Desktop\SQ\msedgedriver.exe"

service = Service(edge_driver_path)
driver = webdriver.Edge(service=service)

url = 'https://www.marketscreener.com/stock-exchange/shares/'
driver.get(url)



In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from fake_useragent import UserAgent

# Proxy setup
proxies = {
    "http": "http://e68e0aa65c0a4e818e6ca7ebc6484a46:@api.zyte.com:8011/",
    "https": "http://e68e0aa65c0a4e818e6ca7ebc6484a46:@api.zyte.com:8011/",
}

# CA certificate path for Zyte
verify_cert = "C:/Users/admin/Desktop/SQ/zyte-ca.crt"

# URL for the POST request
url = "https://www.marketscreener.com/async/stock-screener/more"

# Updated Configuration and Parameters
configuration = "c2tvYk9uN0hCeDIzd25sNEJQdXM4NTBrSmZDL1hzRTNjNmNYQkRlMm1nU0pjTnlJVTdwZ0oxRm1DcmZHSnpDYXZpZUlYb09rSFRwZnJ3b3c3ZEtCdCt2bTBtTmNBZ2N4b2VnWFB0SHQ2NlNPczgzWXRwR3QrbmZLeVYyUkRDMzk2SGZDWk9lOFN0NC9nOEVvUXpJbXJnPT0"
parameters = "bm1xcDU3ODIzTnd2c3JBOWcvbXFjdHlKWnBNdURpZU9NQ0pOZGJXQXRYYkF2WWNLNlVKNXRhalJNUXArblhRR0ZQa2FVRGhUUkhnSDVrN0hqZ2JiZHZLRk0rSHNsenBpVmp0OEJTSkQyc3F6bldhSElEcEJWdndycnVkVzB5czBUNVJ3dHlUYzVwRXhlMEk2dTN6YmpjMDB4NDhvL3VqMUc1dE81QzRTVVBxT2ZTRWxtSi9PZ1h1N0JrZkp4K0RxYkZPWGJockJVNVJ3cmR1RGRNb0I2MFJBZFZJaENkbkxrOVVJNFJqdE0rTzJLa1VsbUJkOTgyLzlLUVhBQmVyM3VOay9iMkdLckxHa2FuUUs4di95VHc9PQ"
token = "2218f21ea8.iYojwnWnASqROkAiuxEf2oJt7pfRVVqr0oi34gN2-lY.wrgOpxfCZE7Ocw9S6nx-k-oMn-CUHyn_hLv_rTslsTn86GC7JPN0TNBSIw"

# Initialize a list to store the data
all_data = []

# Initialize a random user agent generator
ua = UserAgent()

# Set retry limits
max_retries = 3
retry_delay = 30  # seconds

# Start scraping from a specific page
page = 1

while True:

    
    try:
        # Headers with random user agent
        headers = {
            "User-Agent": ua.random,
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "X-Requested-With": "XMLHttpRequest",
            "Origin": "https://www.marketscreener.com",
            "Referer": f"https://www.marketscreener.com/stock-exchange/shares/?cf={configuration}"  # Added detailed referer
        }

        # Data payload for the POST request
        data = {
            "configuration": configuration,
            "parameters": parameters,
            "token": token,
            "page": str(page)  # Set the page number dynamically
        }

        retries = 0
        while retries < max_retries:
            # Send the POST request using the proxy and certificate for Zyte
            response = requests.post(
                url, headers=headers, data=data, proxies=proxies, verify=verify_cert
            )

            # Check if the request was successful
            if response.status_code == 200:
                json_data = response.json()
                html_content = json_data['data']
                soup = BeautifulSoup(html_content, 'html.parser')
                rows = soup.find_all('tr')

                # If no rows are found, stop the loop (assuming that indicates the end)
                if not rows:
                    print(f"No more data on page {page}. Stopping.")
                    break

                # Extract the relevant information for each row
                for row in rows:
                    try:
                        stock_name = row.find('a', {'class': 'link--blue'})
                        stock_name = stock_name.text.strip() if stock_name else "N/A"

                        price = row.find('span', {'data-field': 'last'})
                        price = price.text.strip() if price else "N/A"

                        capitalization = row.find('td', {'aria-label': 'Capitalization'})
                        capitalization = capitalization.text.strip() if capitalization else "N/A"

                        one_year_change = row.find('span', {'data-field': 'change_pct'})
                        one_year_change = one_year_change.text.strip() if one_year_change else "N/A"

                        sector = row.find('div', {'class': 'txt-inline link'})
                        sector = sector.text.strip() if sector else "N/A"

                        # Extract flag class (e.g., 'flag__fr')
                        flag_icon = row.find('i', {'class': lambda x: x and 'flag__' in x})
                        if flag_icon:
                            flag_class = [cls for cls in flag_icon['class'] if 'flag__' in cls][0]
                            country_code = flag_class.split('__')[-1].upper()
                        else:
                            country_code = "N/A"

                        # Append the extracted data to the list
                        all_data.append({
                            'Stock Name': stock_name,
                            'Price': price,
                            'Capitalization': capitalization,
                            '1-Year Price Change': one_year_change,
                            'Sector': sector,
                            'Country Code': country_code
                        })

                    except Exception as row_error:
                        print(f"Error processing row: {row_error}")

                # Print progress
                print(f"Scraped page {page}")

                # Random delay to avoid getting banned (between 2 and 10 seconds)
                time.sleep(random.uniform(2, 10))

                # Move to the next page
                page += 1
                break

            elif response.status_code == 503:
                retries += 1
                print(f"503 error on page {page}. Retrying {retries}/{max_retries}...")
                time.sleep(retry_delay)

            else:
                print(f"Failed to retrieve data from page {page}. Status code: {response.status_code}")
                break

        if retries == max_retries:
            print(f"Failed to retrieve page {page} after {max_retries} retries. Stopping.")
            break

    except Exception as e:
        # Log the exception and stop the process
        print(f"An error occurred: {e}. Stopping on page {page}.")
        break

# Convert the list to a DataFrame
df = pd.DataFrame(all_data)

# Save the DataFrame to an Excel file
df.to_excel('after50.xlsx', index=False)

print("Scraping completed and data saved to 'scraped_data_up_to_page_3.xlsx'")


Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Scraped page 21
Scraped page 22
Scraped page 23
Scraped page 24
Scraped page 25
Scraped page 26
Scraped page 27
Scraped page 28
Scraped page 29
Scraped page 30
Scraped page 31
Scraped page 32
Scraped page 33
Scraped page 34
Scraped page 35
Scraped page 36
Scraped page 37
Scraped page 38
Scraped page 39
Scraped page 40
Scraped page 41
Scraped page 42
Scraped page 43
Scraped page 44
Scraped page 45
Scraped page 46
Scraped page 47
Scraped page 48
Scraped page 49
Scraped page 50
Scraped page 51
Scraped page 52
Scraped page 53
Scraped page 54
Scraped page 55
Scraped page 56
Scraped page 57
Scraped page 58
Scraped page 59
Scraped page 60
Scraped page 61
Scraped page 62
Scraped page 63
S

KeyboardInterrupt: 

In [5]:
df = pd.DataFrame(all_data)

# Save the DataFrame to an Excel file
df.to_excel('after50.xlsx', index=False)

print("Scraping completed and data saved to 'scraped_data_up_to_page_3.xlsx'")

Scraping completed and data saved to 'scraped_data_up_to_page_3.xlsx'
